In [1]:
from pathlib import Path
import pandas as pd
import pickle
import numpy as np

In [2]:
DATA_FOLDER = 'data/'

In [3]:
current_dir = Path.cwd()
relative_path = DATA_FOLDER + 'raw_combined_data.pkl'
frame_path = current_dir.joinpath(relative_path)
shots_df = pd.read_pickle(str(frame_path))

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160032 entries, 0 to 160031
Data columns (total 45 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             160032 non-null  object 
 1   season              160032 non-null  object 
 2   type                160032 non-null  object 
 3   away_code           160032 non-null  object 
 4   home_code           160032 non-null  object 
 5   venue               160032 non-null  object 
 6   period              160032 non-null  int64  
 7   time_elapsed        160032 non-null  object 
 8   cum_time_elapsed    160032 non-null  int64  
 9   period_ord          160032 non-null  object 
 10  is_overtime         160032 non-null  object 
 11  home_score          160032 non-null  int64  
 12  away_score          160032 non-null  int64  
 13  event               160032 non-null  object 
 14  event_team_code     160032 non-null  object 
 15  event_team_is_home  160032 non-nul

In [6]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,is_penalty_shot,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,miss_type,shot_type,is_shootout,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...",False,"{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,False,6,6,3,3,False,False
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,-37.0,-10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...",False,"{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,False,6,6,3,3,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...",False,"{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,False,6,6,3,3,False,False
3,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,02:00,120,1st,REGULAR,0,0,BLOCK,TOR,True,56.0,15.0,None,False,EV,18:00,"{'shot_dist': None, 'event_zone': 'Def. Zone',...",False,"{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 2, 'R': 1, 'D': 2, 'G': 1}",2018020001,1080,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,NaN,Def. Zone,Block,Snap,False,6,6,3,3,False,False
4,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:16,196,1st,REGULAR,0,0,BLOCK,TOR,True,72.0,-6.0,None,False,EV,16:44,"{'shot_dist': None, 'event_zone': 'Def. Zone',...",False,"{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 2, 'L': 1, 'D': 2, 'G': 1}",2018020001,1004,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,NaN,Def. Zone,Block,Slap,False,6,6,3,3,False,False


There are a handful of shots which seem to appear in the html play-by-play data, but not in the game live feed. This can occur when postgame evaluation of the play-by-play makes different determinations as to whether events were shots. By definition, none of these can be goals (which will automatically be classified as a shot-on-goal). There are few enough of theses that omitting them shouldn't be a concern.

In [7]:
shots_df = shots_df[shots_df['is_penalty_shot']==False]
shots_df = shots_df[shots_df['is_shootout']==False]

In [8]:
shots_df.drop(['is_penalty_shot', 'is_shootout'], axis=1, inplace=True)

In [9]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157363 entries, 0 to 160031
Data columns (total 43 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             157363 non-null  object 
 1   season              157363 non-null  object 
 2   type                157363 non-null  object 
 3   away_code           157363 non-null  object 
 4   home_code           157363 non-null  object 
 5   venue               157363 non-null  object 
 6   period              157363 non-null  int64  
 7   time_elapsed        157363 non-null  object 
 8   cum_time_elapsed    157363 non-null  int64  
 9   period_ord          157363 non-null  object 
 10  is_overtime         157363 non-null  object 
 11  home_score          157363 non-null  int64  
 12  away_score          157363 non-null  int64  
 13  event               157363 non-null  object 
 14  event_team_code     157363 non-null  object 
 15  event_team_is_home  157363 non-nul

In [10]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,miss_type,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,-37.0,-10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False
3,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,02:00,120,1st,REGULAR,0,0,BLOCK,TOR,True,56.0,15.0,None,False,EV,18:00,"{'shot_dist': None, 'event_zone': 'Def. Zone',...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 2, 'R': 1, 'D': 2, 'G': 1}",2018020001,1080,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,NaN,Def. Zone,Block,Snap,6,6,3,3,False,False
4,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:16,196,1st,REGULAR,0,0,BLOCK,TOR,True,72.0,-6.0,None,False,EV,16:44,"{'shot_dist': None, 'event_zone': 'Def. Zone',...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 2, 'L': 1, 'D': 2, 'G': 1}",2018020001,1004,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,NaN,Def. Zone,Block,Slap,6,6,3,3,False,False


A few events from one game are missing data as to which players are on the ice. These have additional oddities, as they seem to be near copies of each other, with only minor differences. All events are also indicated as taking place with no time off the clock at the start of the game. It should be safe to ignore these.

In [11]:
missing_players = shots_df['players_h'].isna() | shots_df['players_a'].isna()

In [12]:
shots_df[missing_players]

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,miss_type,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime


In [13]:
shots_df = shots_df[~missing_players]

In [14]:
missing_players = shots_df['players_h'].isna() | shots_df['players_a'].isna()
shots_df[missing_players]

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,miss_type,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime


There are a handful of remaining fields with null entries. The handful of `event_coord_x` and `event_coord_y` will be imputed with medians (the mean for `event_coord_x` will be skewed by outliers corresponding to shots from the opposing zone). The remaining fields with missing entries are `secondary_type`, `shot_dist`, `event_zone`, `miss_type`, and `shot_type`. 

Of these, `shot_dist` is known to be missing when the event type is `BLOCK`. The `miss_type` is missing when the event type is `GOAL`.

In [15]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157363 entries, 0 to 160031
Data columns (total 43 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             157363 non-null  object 
 1   season              157363 non-null  object 
 2   type                157363 non-null  object 
 3   away_code           157363 non-null  object 
 4   home_code           157363 non-null  object 
 5   venue               157363 non-null  object 
 6   period              157363 non-null  int64  
 7   time_elapsed        157363 non-null  object 
 8   cum_time_elapsed    157363 non-null  int64  
 9   period_ord          157363 non-null  object 
 10  is_overtime         157363 non-null  object 
 11  home_score          157363 non-null  int64  
 12  away_score          157363 non-null  int64  
 13  event               157363 non-null  object 
 14  event_team_code     157363 non-null  object 
 15  event_team_is_home  157363 non-nul

First, examine the `BLOCK` event and `shot_dist`. Blocks are problematic as there is no way to determine the location that the shot was taken from.

In [16]:
shots_df.event.value_counts()

SHOT     77828
BLOCK    39083
MISS     32408
GOAL      8044
Name: event, dtype: int64

In [17]:
shots_df.shot_dist.isna().sum()

39083

In [18]:
shots_df = shots_df[shots_df.event != 'BLOCK']

In [19]:
shots_df.event.value_counts()

SHOT    77828
MISS    32408
GOAL     8044
Name: event, dtype: int64

In [20]:
shots_df.shot_dist.isna().sum()

0

In [21]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 43 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             118280 non-null  object 
 1   season              118280 non-null  object 
 2   type                118280 non-null  object 
 3   away_code           118280 non-null  object 
 4   home_code           118280 non-null  object 
 5   venue               118280 non-null  object 
 6   period              118280 non-null  int64  
 7   time_elapsed        118280 non-null  object 
 8   cum_time_elapsed    118280 non-null  int64  
 9   period_ord          118280 non-null  object 
 10  is_overtime         118280 non-null  object 
 11  home_score          118280 non-null  int64  
 12  away_score          118280 non-null  int64  
 13  event               118280 non-null  object 
 14  event_team_code     118280 non-null  object 
 15  event_team_is_home  118280 non-nul

In [22]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,miss_type,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,-37.0,-10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False


From construction, `miss_type` should only be missing when the event type is `GOAL`. There to turn out to be three miss events where `miss_type` is null. These can simply be coded as miss. The remaining `miss_type` will be coded to `Save` and the column will be renamed to reflect that it is the result of the shot-attempt as opposed to the reason for a miss.

In [23]:
shots_df['miss_type'].value_counts()

Save            77828
Wide of Net     27264
Over Net         3253
Goalpost         1565
Hit Crossbar      326
Name: miss_type, dtype: int64

In [24]:
shots_df[shots_df['miss_type'].isna()]['event'].value_counts()

GOAL    8044
Name: event, dtype: int64

In [25]:
shots_df.loc[shots_df['miss_type'].isna(), 'miss_type'] = shots_df['event'].str.title()

In [26]:
shots_df['miss_type'].value_counts()

Save            77828
Wide of Net     27264
Goal             8044
Over Net         3253
Goalpost         1565
Hit Crossbar      326
Name: miss_type, dtype: int64

In [27]:
shots_df.loc[shots_df['miss_type']=='Miss','miss_type'] = 'Save'

In [28]:
shots_df['miss_type'].value_counts()

Save            77828
Wide of Net     27264
Goal             8044
Over Net         3253
Goalpost         1565
Hit Crossbar      326
Name: miss_type, dtype: int64

In [29]:
shots_df.rename({'miss_type': 'result'}, axis=1, inplace=True)

In [30]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 43 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             118280 non-null  object 
 1   season              118280 non-null  object 
 2   type                118280 non-null  object 
 3   away_code           118280 non-null  object 
 4   home_code           118280 non-null  object 
 5   venue               118280 non-null  object 
 6   period              118280 non-null  int64  
 7   time_elapsed        118280 non-null  object 
 8   cum_time_elapsed    118280 non-null  int64  
 9   period_ord          118280 non-null  object 
 10  is_overtime         118280 non-null  object 
 11  home_score          118280 non-null  int64  
 12  away_score          118280 non-null  int64  
 13  event               118280 non-null  object 
 14  event_team_code     118280 non-null  object 
 15  event_team_is_home  118280 non-nul

In [31]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,result,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,-37.0,-10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False


Make sure attacking direction is toward positive x-values.

In [32]:
shots_df['is_attacking_positive'] = (shots_df['event_coord_x'] >= 0)   

In [33]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 44 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   game_id                118280 non-null  object 
 1   season                 118280 non-null  object 
 2   type                   118280 non-null  object 
 3   away_code              118280 non-null  object 
 4   home_code              118280 non-null  object 
 5   venue                  118280 non-null  object 
 6   period                 118280 non-null  int64  
 7   time_elapsed           118280 non-null  object 
 8   cum_time_elapsed       118280 non-null  int64  
 9   period_ord             118280 non-null  object 
 10  is_overtime            118280 non-null  object 
 11  home_score             118280 non-null  int64  
 12  away_score             118280 non-null  int64  
 13  event                  118280 non-null  object 
 14  event_team_code        118280 non-nu

In [34]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,result,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime,is_attacking_positive
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False,True
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,-37.0,-10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False,True
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False,True
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False,True


As should be anticipated, the attacking direction is positive roughly half the time.

In [35]:
shots_df.is_attacking_positive.value_counts()

False    59149
True     59131
Name: is_attacking_positive, dtype: int64

In [36]:
positive_grouping = shots_df.groupby(['game_id', 'period', 'event_team_is_home'])['is_attacking_positive'].mean()
positive_grouping = positive_grouping.to_frame(name='coord_accuracy')
positive_grouping

coord_accuracy
game_id    period event_team_is_home                
2018020001 1      False                     1.000000
                  True                      0.000000
           2      False                     0.000000
                  True                      1.000000
           3      False                     1.000000
...                                              ...
2018030417 1      True                      1.000000
           2      False                     0.888889
                  True                      0.166667
           3      False                     0.000000
                  True                      1.000000

[8647 rows x 1 columns]

In [37]:
positive_grouping.describe()

,coord_accuracy
count,8647.000000
mean,0.499206
std,0.478330
min,0.000000
25%,0.000000
50%,0.333333
75%,1.000000
max,1.000000


In [38]:
shots_df = shots_df.merge(positive_grouping, left_on=['game_id', 'period', 'event_team_is_home'], right_index=True)

In [39]:
shots_df['event_coord_x'] = np.where(shots_df.coord_accuracy < 0.5, -shots_df.event_coord_x, shots_df.event_coord_x)
shots_df['event_coord_y'] = np.where(shots_df.coord_accuracy < 0.5, -shots_df.event_coord_y, shots_df.event_coord_y)

In [40]:
shots_df['is_attacking_positive'] = (shots_df['event_coord_x'] >= 0)

In [41]:
shots_df.is_attacking_positive.value_counts()

True     115581
False      2699
Name: is_attacking_positive, dtype: int64

In [42]:
positive_grouping = shots_df.groupby(['game_id', 'period', 'event_team_is_home'])['is_attacking_positive'].mean()
positive_grouping = positive_grouping.to_frame(name='coord_accuracy')
positive_grouping.describe()

,coord_accuracy
count,8647.000000
mean,0.976037
std,0.048976
min,0.500000
25%,0.960000
50%,1.000000
75%,1.000000
max,1.000000


In [43]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160026
Data columns (total 45 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   game_id                118280 non-null  object 
 1   season                 118280 non-null  object 
 2   type                   118280 non-null  object 
 3   away_code              118280 non-null  object 
 4   home_code              118280 non-null  object 
 5   venue                  118280 non-null  object 
 6   period                 118280 non-null  int64  
 7   time_elapsed           118280 non-null  object 
 8   cum_time_elapsed       118280 non-null  int64  
 9   period_ord             118280 non-null  object 
 10  is_overtime            118280 non-null  object 
 11  home_score             118280 non-null  int64  
 12  away_score             118280 non-null  int64  
 13  event                  118280 non-null  object 
 14  event_team_code        118280 non-nu

In [44]:
shots_df.drop(['is_attacking_positive', 'coord_accuracy'], axis=1, inplace=True)

In [45]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160026
Data columns (total 43 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             118280 non-null  object 
 1   season              118280 non-null  object 
 2   type                118280 non-null  object 
 3   away_code           118280 non-null  object 
 4   home_code           118280 non-null  object 
 5   venue               118280 non-null  object 
 6   period              118280 non-null  int64  
 7   time_elapsed        118280 non-null  object 
 8   cum_time_elapsed    118280 non-null  int64  
 9   period_ord          118280 non-null  object 
 10  is_overtime         118280 non-null  object 
 11  home_score          118280 non-null  int64  
 12  away_score          118280 non-null  int64  
 13  event               118280 non-null  object 
 14  event_team_code     118280 non-null  object 
 15  event_team_is_home  118280 non-nul

In [46]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,result,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False
7,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,04:01,241,1st,REGULAR,0,0,MISS,MTL,False,81.0,-4.0,None,False,EV,15:59,"{'shot_dist': 10, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,959,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,10.0,Off. Zone,Wide of Net,Tip-In,6,6,3,3,False,False


In [47]:
shots_df.sort_index(inplace=True)
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,result,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,37.0,10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False


Change the viewpoint to frame columns as attacking/defending teams instead of home/away teams.

In [48]:
shots_df['event_team_is_home'] = shots_df['event_team_is_home'].astype(bool)

In [49]:
shots_df.loc[shots_df.event_team_is_home, 'att_players'] = shots_df['players_h']
shots_df.loc[~shots_df.event_team_is_home, 'att_players'] = shots_df['players_a']
shots_df.loc[~shots_df.event_team_is_home, 'def_players'] = shots_df['players_h']
shots_df.loc[shots_df.event_team_is_home, 'def_players'] = shots_df['players_a']

In [50]:
shots_df.loc[shots_df.event_team_is_home, 'att_skaters'] = shots_df['skaters_h']
shots_df.loc[~shots_df.event_team_is_home, 'att_skaters'] = shots_df['skaters_a']
shots_df.loc[~shots_df.event_team_is_home, 'def_skaters'] = shots_df['skaters_h']
shots_df.loc[shots_df.event_team_is_home, 'def_skaters'] = shots_df['skaters_a']

In [51]:
shots_df.loc[shots_df.event_team_is_home, 'att_forwards'] = shots_df['fwds_h']
shots_df.loc[~shots_df.event_team_is_home, 'att_forwards'] = shots_df['fwds_a']
shots_df.loc[~shots_df.event_team_is_home, 'def_forwards'] = shots_df['fwds_h']
shots_df.loc[shots_df.event_team_is_home, 'def_forwards'] = shots_df['fwds_a']

In [52]:
shots_df.loc[shots_df.event_team_is_home, 'is_extra_attacker'] = shots_df['goalie_pulled_h']
shots_df.loc[~shots_df.event_team_is_home, 'is_extra_attacker'] = shots_df['goalie_pulled_a']
shots_df.loc[~shots_df.event_team_is_home, 'is_empty_net'] = shots_df['goalie_pulled_h']
shots_df.loc[shots_df.event_team_is_home, 'is_empty_net'] = shots_df['goalie_pulled_a']

In [53]:
shots_df.loc[shots_df.event_team_is_home, 'att_score'] = shots_df['home_score']
shots_df.loc[~shots_df.event_team_is_home, 'att_score'] = shots_df['away_score']
shots_df.loc[~shots_df.event_team_is_home, 'def_score'] = shots_df['home_score']
shots_df.loc[shots_df.event_team_is_home, 'def_score'] = shots_df['away_score']

In [54]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 53 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             118280 non-null  object 
 1   season              118280 non-null  object 
 2   type                118280 non-null  object 
 3   away_code           118280 non-null  object 
 4   home_code           118280 non-null  object 
 5   venue               118280 non-null  object 
 6   period              118280 non-null  int64  
 7   time_elapsed        118280 non-null  object 
 8   cum_time_elapsed    118280 non-null  int64  
 9   period_ord          118280 non-null  object 
 10  is_overtime         118280 non-null  object 
 11  home_score          118280 non-null  int64  
 12  away_score          118280 non-null  int64  
 13  event               118280 non-null  object 
 14  event_team_code     118280 non-null  object 
 15  event_team_is_home  118280 non-nul

In [55]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,result,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime,att_players,def_players,att_skaters,def_skaters,att_forwards,def_forwards,is_extra_attacker,is_empty_net,att_score,def_score
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,37.0,10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0


The indicated score is after the play in question, while this investigation is concerned with the score when the shot was first taken. As a result, the attacking team score should be decremented by one for events where a goal was scored.

In [56]:
shots_df['att_score'] = shots_df['att_score'] - (shots_df.event == 'GOAL')

Now that scores are correct, calculate the attacking team's lead at the time of the event (negative values imply the attacking team is behind).

In [57]:
shots_df['lead_size'] = shots_df['att_score'] - shots_df['def_score']

In [58]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,home_score,away_score,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,desc,pos_a,pos_h,game_id_htmlreport,seconds_remaining,fwd_def_a,fwd_def_h,skaters_a,skaters_h,goalie_pulled_a,goalie_pulled_h,shot_dist,event_zone,result,shot_type,players_h,players_a,fwds_h,fwds_a,is_playoff_game,is_overtime,att_players,def_players,att_skaters,def_skaters,att_forwards,def_forwards,is_extra_attacker,is_empty_net,att_score,def_score,lead_size
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,0,0,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,"{'shot_dist': 22, 'event_zone': 'Off. Zone', '...","{'C': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1171,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,22.0,Off. Zone,Save,Backhand,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,0,0,SHOT,TOR,True,37.0,10.0,Snap Shot,False,EV,19:11,"{'shot_dist': 52, 'event_zone': 'Off. Zone', '...","{'C': 1, 'R': 1, 'L': 1, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1151,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,52.0,Off. Zone,Save,Snap,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,0,0,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,"{'shot_dist': 48, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,1140,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,48.0,Off. Zone,Save,Wrist,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,0,0,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,"{'shot_dist': 27, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,971,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,27.0,Off. Zone,Wide of Net,Snap,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,0,0,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,"{'shot_dist': 39, 'event_zone': 'Off. Zone', '...","{'R': 1, 'L': 2, 'D': 2, 'G': 1}","{'C': 3, 'D': 2, 'G': 1}",2018020001,966,"{'FWD': 3, 'DEF': 2, 'GOAL': 1}","{'FWD': 3, 'DEF': 2, 'GOAL': 1}",5,5,False,False,39.0,Off. Zone,Save,Snap,6,6,3,3,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0


In [59]:
shots_df.drop(['home_score', 'away_score', 'desc', 'pos_a', 'pos_h', 'game_id_htmlreport', 'fwd_def_a', 'fwd_def_h', \
              'skaters_a', 'skaters_h', 'goalie_pulled_a', 'goalie_pulled_h', 'players_h', 'players_a', \
               'fwds_h', 'fwds_a'], axis=1, inplace=True)

In [60]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 38 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   game_id             118280 non-null  object 
 1   season              118280 non-null  object 
 2   type                118280 non-null  object 
 3   away_code           118280 non-null  object 
 4   home_code           118280 non-null  object 
 5   venue               118280 non-null  object 
 6   period              118280 non-null  int64  
 7   time_elapsed        118280 non-null  object 
 8   cum_time_elapsed    118280 non-null  int64  
 9   period_ord          118280 non-null  object 
 10  is_overtime         118280 non-null  object 
 11  event               118280 non-null  object 
 12  event_team_code     118280 non-null  object 
 13  event_team_is_home  118280 non-null  bool   
 14  event_coord_x       118277 non-null  float64
 15  event_coord_y       118278 non-nul

In [61]:
shots_df.head()

,game_id,season,type,away_code,home_code,venue,period,time_elapsed,cum_time_elapsed,period_ord,is_overtime,event,event_team_code,event_team_is_home,event_coord_x,event_coord_y,secondary_type,is_rebound,strength,time_remaining,seconds_remaining,shot_dist,event_zone,result,shot_type,is_playoff_game,is_overtime,att_players,def_players,att_skaters,def_skaters,att_forwards,def_forwards,is_extra_attacker,is_empty_net,att_score,def_score,lead_size
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:29,29,1st,REGULAR,SHOT,MTL,False,78.0,-19.0,Backhand,False,EV,19:31,1171,22.0,Off. Zone,Save,Backhand,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,00:49,49,1st,REGULAR,SHOT,TOR,True,37.0,10.0,Snap Shot,False,EV,19:11,1151,52.0,Off. Zone,Save,Snap,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,01:00,60,1st,REGULAR,SHOT,MTL,False,47.0,-23.0,Wrist Shot,False,EV,19:00,1140,48.0,Off. Zone,Save,Wrist,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:49,229,1st,REGULAR,MISS,MTL,False,73.0,22.0,None,False,EV,16:11,971,27.0,Off. Zone,Wide of Net,Snap,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,1,03:54,234,1st,REGULAR,SHOT,MTL,False,53.0,14.0,Snap Shot,False,EV,16:06,966,39.0,Off. Zone,Save,Snap,False,False,6.0,6.0,5.0,5.0,3.0,3.0,False,False,0.0,0.0,0.0


In [62]:
shots_df.rename({'event_team_is_home': 'is_home'}, axis=1, inplace=True)

In [63]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 38 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   game_id            118280 non-null  object 
 1   season             118280 non-null  object 
 2   type               118280 non-null  object 
 3   away_code          118280 non-null  object 
 4   home_code          118280 non-null  object 
 5   venue              118280 non-null  object 
 6   period             118280 non-null  int64  
 7   time_elapsed       118280 non-null  object 
 8   cum_time_elapsed   118280 non-null  int64  
 9   period_ord         118280 non-null  object 
 10  is_overtime        118280 non-null  object 
 11  event              118280 non-null  object 
 12  event_team_code    118280 non-null  object 
 13  is_home            118280 non-null  bool   
 14  event_coord_x      118277 non-null  float64
 15  event_coord_y      118278 non-null  float64
 16  se

In [64]:
shots_df['is_overtime'] = shots_df['is_overtime'].astype(bool)
shots_df['is_extra_attacker'] = shots_df['is_extra_attacker'].astype(bool)
shots_df['is_empty_net'] = shots_df['is_empty_net'].astype(bool)

In [65]:
shots_df['att_players'] = shots_df['att_players'].astype(int)
shots_df['def_players'] = shots_df['def_players'].astype(int)
shots_df['att_skaters'] = shots_df['att_skaters'].astype(int)
shots_df['def_skaters'] = shots_df['def_skaters'].astype(int)
shots_df['att_forwards'] = shots_df['att_forwards'].astype(int)
shots_df['def_forwards'] = shots_df['def_forwards'].astype(int)
shots_df['att_score'] = shots_df['att_score'].astype(int)
shots_df['def_score'] = shots_df['def_score'].astype(int)
shots_df['lead_size'] = shots_df['lead_size'].astype(int)

In [66]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 38 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   game_id            118280 non-null  object 
 1   season             118280 non-null  object 
 2   type               118280 non-null  object 
 3   away_code          118280 non-null  object 
 4   home_code          118280 non-null  object 
 5   venue              118280 non-null  object 
 6   period             118280 non-null  int64  
 7   time_elapsed       118280 non-null  object 
 8   cum_time_elapsed   118280 non-null  int64  
 9   period_ord         118280 non-null  object 
 10  is_overtime        118280 non-null  bool   
 11  event              118280 non-null  object 
 12  event_team_code    118280 non-null  object 
 13  is_home            118280 non-null  bool   
 14  event_coord_x      118277 non-null  float64
 15  event_coord_y      118278 non-null  float64
 16  se

In [67]:
shots_df.select_dtypes(include=['object']).head()

,game_id,season,type,away_code,home_code,venue,time_elapsed,period_ord,event,event_team_code,secondary_type,strength,time_remaining,event_zone,result,shot_type
0,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,00:29,1st,SHOT,MTL,Backhand,EV,19:31,Off. Zone,Save,Backhand
1,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,00:49,1st,SHOT,TOR,Snap Shot,EV,19:11,Off. Zone,Save,Snap
2,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,01:00,1st,SHOT,MTL,Wrist Shot,EV,19:00,Off. Zone,Save,Wrist
5,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,03:49,1st,MISS,MTL,None,EV,16:11,Off. Zone,Wide of Net,Snap
6,2018020001,20182019,R,MTL,TOR,Scotiabank Arena,03:54,1st,SHOT,MTL,Snap Shot,EV,16:06,Off. Zone,Save,Snap


In [68]:
shots_df.rename({'event_team_code': 'att_code'}, axis=1, inplace=True)

In [69]:
shots_df.loc[shots_df.is_home, 'def_code'] = shots_df['away_code']
shots_df.loc[~shots_df.is_home, 'def_code'] = shots_df['home_code']

In [70]:
shots_df.drop(['type', 'away_code', 'home_code', 'time_elapsed', 'period_ord', 'time_remaining'], axis=1, inplace=True)

In [71]:
shots_df.select_dtypes(include=['object']).head()

,game_id,season,venue,event,att_code,secondary_type,strength,event_zone,result,shot_type,def_code
0,2018020001,20182019,Scotiabank Arena,SHOT,MTL,Backhand,EV,Off. Zone,Save,Backhand,TOR
1,2018020001,20182019,Scotiabank Arena,SHOT,TOR,Snap Shot,EV,Off. Zone,Save,Snap,MTL
2,2018020001,20182019,Scotiabank Arena,SHOT,MTL,Wrist Shot,EV,Off. Zone,Save,Wrist,TOR
5,2018020001,20182019,Scotiabank Arena,MISS,MTL,None,EV,Off. Zone,Wide of Net,Snap,TOR
6,2018020001,20182019,Scotiabank Arena,SHOT,MTL,Snap Shot,EV,Off. Zone,Save,Snap,TOR


In [72]:
shots_df.select_dtypes(include=['int', 'int64']).head()

,period,cum_time_elapsed,seconds_remaining,att_players,def_players,att_skaters,def_skaters,att_forwards,def_forwards,att_score,def_score,lead_size
0,1,29,1171,6,6,5,5,3,3,0,0,0
1,1,49,1151,6,6,5,5,3,3,0,0,0
2,1,60,1140,6,6,5,5,3,3,0,0,0
5,1,229,971,6,6,5,5,3,3,0,0,0
6,1,234,966,6,6,5,5,3,3,0,0,0


In [73]:
shots_df.select_dtypes(include=['float']).head()

,event_coord_x,event_coord_y,shot_dist
0,78.0,-19.0,22.0
1,37.0,10.0,52.0
2,47.0,-23.0,48.0
5,73.0,22.0,27.0
6,53.0,14.0,39.0


In [74]:
shots_df.select_dtypes(include=['bool']).head()

,is_overtime,is_home,is_rebound,is_playoff_game,is_overtime,is_extra_attacker,is_empty_net
0,True,False,False,False,False,False,False
1,True,True,False,False,False,False,False
2,True,False,False,False,False,False,False
5,True,False,False,False,False,False,False
6,True,False,False,False,False,False,False


Data types now appear to be as expected. While each of the float fields could also be cast as integers with no change, they all represent physical distances and will be kept as floats. 

Players on ice are occasionally tracked incorrectly. In particular, there can never be more than 6 or fewer than 4 players on ice for any given team. Checking the columns which track player counts, it's apparent that both conditions are violated by some data rows. 

In [75]:
player_count_columns = ['att_players', 'def_players', 'att_skaters', 'def_skaters', 'att_forwards', 'def_forwards'] 
shots_df[player_count_columns].describe()

,att_players,def_players,att_skaters,def_skaters,att_forwards,def_forwards
count,118280.000000,118280.000000,118280.000000,118280.000000,118280.000000,118280.000000
mean,5.939609,5.826217,4.959773,4.832229,3.084258,2.860501
std,0.278343,0.418836,0.313309,0.427841,0.442982,0.444620
min,4.000000,2.000000,3.000000,1.000000,1.000000,0.000000
25%,6.000000,6.000000,5.000000,5.000000,3.000000,3.000000
50%,6.000000,6.000000,5.000000,5.000000,3.000000,3.000000
75%,6.000000,6.000000,5.000000,5.000000,3.000000,3.000000
max,7.000000,7.000000,7.000000,6.000000,6.000000,6.000000


Any given error in player counts calls the correctness of the other count columns into question. These will be converted to null and then imputed based on the game situation. Since the skaters and forwards columns are less restrictive (skaters can vary between 3 and 6; forwards between 0 and 6), it makes sense to key on the the fields `att_players` and `def_players`. Whenever these are outside the allowable range, all six columns will be set to null.

In [76]:
invalid_player_count = (shots_df['att_players'] > 6) | (shots_df['att_players'] < 4) \
                        | (shots_df['def_players'] > 6) | (shots_df['def_players'] < 4)
shots_df.loc[invalid_player_count, player_count_columns] = np.NaN

This changes the data type of these columns to float, since the default integer-type is non-nullable. The type can be changed back to an int-type using `Int64` (capital 'I'). This is an experimental nullable integer data type (see https://pandas.pydata.org/docs/user_guide/integer_na.html).

In [77]:
shots_df[player_count_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   att_players   118268 non-null  float64
 1   def_players   118268 non-null  float64
 2   att_skaters   118268 non-null  float64
 3   def_skaters   118268 non-null  float64
 4   att_forwards  118268 non-null  float64
 5   def_forwards  118268 non-null  float64
dtypes: float64(6)
memory usage: 10.3 MB


In [78]:
shots_df[player_count_columns] = shots_df[player_count_columns].astype('Int64')
shots_df[player_count_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   att_players   118268 non-null  Int64
 1   def_players   118268 non-null  Int64
 2   att_skaters   118268 non-null  Int64
 3   def_skaters   118268 non-null  Int64
 4   att_forwards  118268 non-null  Int64
 5   def_forwards  118268 non-null  Int64
dtypes: Int64(6)
memory usage: 11.0 MB


In [79]:
shots_df[player_count_columns].describe()

,att_players,def_players,att_skaters,def_skaters,att_forwards,def_forwards
count,118268.000000,118268.000000,118268.000000,118268.000000,118268.000000,118268.000000
mean,5.939527,5.826259,4.959710,4.832271,3.084173,2.860529
std,0.278203,0.418524,0.313196,0.427537,0.442879,0.444511
min,4.000000,4.000000,3.000000,3.000000,1.000000,1.000000
25%,6.000000,6.000000,5.000000,5.000000,3.000000,3.000000
50%,6.000000,6.000000,5.000000,5.000000,3.000000,3.000000
75%,6.000000,6.000000,5.000000,5.000000,3.000000,3.000000
max,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000


In [80]:
shots_df['is_goal'] = shots_df['event'] == 'GOAL'

The field name `is_overtime` was accidentally repeated in the previous workbook. It is necessary to get rid of the incorrect copy, so it is renamed and dropped.

In [81]:
shots_df['is_overtime']

,is_overtime,is_overtime
0,True,False
1,True,False
2,True,False
5,True,False
6,True,False
...,...,...
160026,True,False
160027,True,False
160028,True,False
160029,True,False


In [82]:
duplicated_columns_list = []
list_of_all_columns = list(shots_df.columns)
for column in list_of_all_columns:
    if list_of_all_columns.count(column) > 1 and not column in duplicated_columns_list:
        duplicated_columns_list.append(column)
duplicated_columns_list

['is_overtime']

In [83]:
for column in duplicated_columns_list:
    list_of_all_columns[list_of_all_columns.index(column)] = column + '_err'
    list_of_all_columns[list_of_all_columns.index(column)] = column

In [84]:
list_of_all_columns

['game_id',
 'season',
 'venue',
 'period',
 'cum_time_elapsed',
 'is_overtime_err',
 'event',
 'att_code',
 'is_home',
 'event_coord_x',
 'event_coord_y',
 'secondary_type',
 'is_rebound',
 'strength',
 'seconds_remaining',
 'shot_dist',
 'event_zone',
 'result',
 'shot_type',
 'is_playoff_game',
 'is_overtime',
 'att_players',
 'def_players',
 'att_skaters',
 'def_skaters',
 'att_forwards',
 'def_forwards',
 'is_extra_attacker',
 'is_empty_net',
 'att_score',
 'def_score',
 'lead_size',
 'def_code',
 'is_goal']

In [85]:
shots_df.columns = list_of_all_columns

In [86]:
shots_df.drop('is_overtime_err', axis=1, inplace=True)

Add fields to compare the shot distance in the html play-by-play with that derived from the coordinates from the live feed.

In [87]:
shots_df['calc_dist'] = np.sqrt((shots_df['event_coord_x']-89)**2 + shots_df['event_coord_y']**2)
shots_df['dist_difference'] = np.abs(shots_df['calc_dist'] - shots_df['shot_dist'])

In [88]:
shots_df = shots_df[['game_id', 'season', 'is_playoff_game', 'venue', 'att_code', 'def_code', 'is_home',
                     'period', 'cum_time_elapsed', 'is_overtime', 'seconds_remaining', 'strength', 
                     'att_score', 'def_score', 'lead_size',
                     'event', 'att_players', 'att_skaters', 'att_forwards', 'def_players', 'def_skaters', 'def_forwards',
                     'is_extra_attacker', 'is_empty_net',
                     'event_zone', 'shot_type', 'secondary_type', 
                     'event_coord_x', 'event_coord_y', 'is_rebound', 'shot_dist', 'calc_dist', 'dist_difference',
                     'result', 'is_goal']]

In [89]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 35 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   game_id            118280 non-null  object 
 1   season             118280 non-null  object 
 2   is_playoff_game    118280 non-null  bool   
 3   venue              118280 non-null  object 
 4   att_code           118280 non-null  object 
 5   def_code           118280 non-null  object 
 6   is_home            118280 non-null  bool   
 7   period             118280 non-null  int64  
 8   cum_time_elapsed   118280 non-null  int64  
 9   is_overtime        118280 non-null  bool   
 10  seconds_remaining  118280 non-null  int64  
 11  strength           118280 non-null  object 
 12  att_score          118280 non-null  int32  
 13  def_score          118280 non-null  int32  
 14  lead_size          118280 non-null  int32  
 15  event              118280 non-null  object 
 16  at

In [90]:
shots_df.head()

,game_id,season,is_playoff_game,venue,att_code,def_code,is_home,period,cum_time_elapsed,is_overtime,seconds_remaining,strength,att_score,def_score,lead_size,event,att_players,att_skaters,att_forwards,def_players,def_skaters,def_forwards,is_extra_attacker,is_empty_net,event_zone,shot_type,secondary_type,event_coord_x,event_coord_y,is_rebound,shot_dist,calc_dist,dist_difference,result,is_goal
0,2018020001,20182019,False,Scotiabank Arena,MTL,TOR,False,1,29,False,1171,EV,0,0,0,SHOT,6,5,3,6,5,3,False,False,Off. Zone,Backhand,Backhand,78.0,-19.0,False,22.0,21.954498,0.045502,Save,False
1,2018020001,20182019,False,Scotiabank Arena,TOR,MTL,True,1,49,False,1151,EV,0,0,0,SHOT,6,5,3,6,5,3,False,False,Off. Zone,Snap,Snap Shot,37.0,10.0,False,52.0,52.952809,0.952809,Save,False
2,2018020001,20182019,False,Scotiabank Arena,MTL,TOR,False,1,60,False,1140,EV,0,0,0,SHOT,6,5,3,6,5,3,False,False,Off. Zone,Wrist,Wrist Shot,47.0,-23.0,False,48.0,47.885280,0.114720,Save,False
5,2018020001,20182019,False,Scotiabank Arena,MTL,TOR,False,1,229,False,971,EV,0,0,0,MISS,6,5,3,6,5,3,False,False,Off. Zone,Snap,None,73.0,22.0,False,27.0,27.202941,0.202941,Wide of Net,False
6,2018020001,20182019,False,Scotiabank Arena,MTL,TOR,False,1,234,False,966,EV,0,0,0,SHOT,6,5,3,6,5,3,False,False,Off. Zone,Snap,Snap Shot,53.0,14.0,False,39.0,38.626416,0.373584,Save,False


In [91]:
current_dir = Path.cwd()
relative_path = DATA_FOLDER + 'cleaned_data.pkl'
frame_path = current_dir.joinpath(relative_path)
shots_df.to_pickle(str(frame_path))

In [92]:
shots_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118280 entries, 0 to 160031
Data columns (total 35 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   game_id            118280 non-null  object 
 1   season             118280 non-null  object 
 2   is_playoff_game    118280 non-null  bool   
 3   venue              118280 non-null  object 
 4   att_code           118280 non-null  object 
 5   def_code           118280 non-null  object 
 6   is_home            118280 non-null  bool   
 7   period             118280 non-null  int64  
 8   cum_time_elapsed   118280 non-null  int64  
 9   is_overtime        118280 non-null  bool   
 10  seconds_remaining  118280 non-null  int64  
 11  strength           118280 non-null  object 
 12  att_score          118280 non-null  int32  
 13  def_score          118280 non-null  int32  
 14  lead_size          118280 non-null  int32  
 15  event              118280 non-null  object 
 16  at